In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression


pd.set_option('display.max_columns', None)

In [2]:
diamond_df = pd.read_csv("../../../archive/blazed_diamond_ranked_10min.csv")
pro_df = pd.read_csv("../../../archive/10_7_pro_games.csv")

In [3]:
diamond_df.head()

,blueWins,blueWardsDestroyed,blueFirstBlood,blueKills,blueAssists,blueDragons,blueHeralds,blueTowersDestroyed,blueAvgLevel,blueTotalJungleMinionsKilled,blueGoldDiff,blueExperienceDiff,blueCSPerMin,blueGoldPerMin,redWardsDestroyed,redKills,redAssists,redDragons,redHeralds,redTowersDestroyed,redAvgLevel,redTotalJungleMinionsKilled,redCSPerMin,redGoldPerMin,blueVisionScore,redVisionScore
0,0,2,1,9,11,0,0,0,6.6,36,643,-8,19.5,1721.0,6,6,8,0,0,0,6.8,55,19.7,1656.7,28,15
1,0,1,0,5,5,0,0,0,6.6,43,-2908,-1173,17.4,1471.2,1,5,2,1,1,1,6.8,52,24.0,1762.0,12,12
2,0,0,0,7,4,1,0,0,6.4,46,-1172,-1033,18.6,1611.3,3,11,14,0,0,0,6.8,28,20.3,1728.5,15,15
3,0,1,0,4,5,0,1,0,7.0,55,-1321,-7,20.1,1515.7,2,5,10,0,0,0,7.0,47,23.5,1647.8,43,15
4,0,4,0,6,6,0,0,0,7.0,57,-1004,230,21.0,1640.0,2,6,7,1,0,0,7.0,67,22.5,1740.4,75,17


In [4]:
pro_df.head()

,Unnamed: 0,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,33540,ESPORTSTMNT06/1190148,complete,http://matchhistory.na.leagueoflegends.com/en/...,EM,2020,Spring,0,2020-04-06 15:11:06,1,10.07,1,Blue,top,Phones,GamerLegion,Wukong,Nunu & Willump,Elise,Senna,Akali,Karthus,1649,1,3,2,6,18,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.6549,0.9096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,14581.0,530.5397,0.264244,709.0115,621.6859,11.0,0.4002,1.0,0.0364,4.0,16.0,0.5822,11592,7952.0,289.3390,0.222016,11450.0,NaN,228.0,226.0,2.0,1.0,0.0,8.2959,3121.0,4007.0,75.0,4025.0,4716.0,84.0,-904.0,-709.0,-9.0,0.0,0.0,1.0,2.0,0.0,0.0,5766.0,7015.0,121.0,6395.0,7086.0,129.0,-629.0,-71.0,-8.0,1.0,0.0,2.0,2.0,0.0,1.0
1,33541,ESPORTSTMNT06/1190148,complete,http://matchhistory.na.leagueoflegends.com/en/...,EM,2020,Spring,0,2020-04-06 15:11:06,1,10.07,2,Blue,jng,Rabble,GamerLegion,Trundle,Nunu & Willump,Elise,Senna,Akali,Karthus,1649,1,2,1,13,18,7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6549,0.9096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,7429.0,270.3093,0.134632,817.2226,830.2486,10.0,0.3639,10.0,0.3639,9.0,31.0,1.1280,9381,5741.0,208.8902,0.160289,7675.0,NaN,124.0,25.0,99.0,48.0,9.0,4.5118,3122.0,3329.0,60.0,3328.0,3297.0,60.0,-206.0,32.0,0.0,0.0,3.0,0.0,1.0,1.0,0.0,4818.0,5153.0,88.0,5030.0,5190.0,96.0,-212.0,-37.0,-8.0,0.0,4.0,0.0,1.0,2.0,0.0
2,33542,ESPORTSTMNT06/1190148,complete,http://matchhistory.na.leagueoflegends.com/en/...,EM,2020,Spring,0,2020-04-06 15:11:06,1,10.07,3,Blue,mid,Nite,GamerLegion,Syndra,Nunu & Willump,Elise,Senna,Akali,Karthus,1649,1,2,1,7,18,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6549,0.9096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,12454.0,453.1474,0.225698,259.3936,133.3899,14.0,0.5094,3.0,0.1092,7.0,43.0,1.5646,11441,7801.0,283.8448,0.217801,9900.0,NaN,254.0,221.0,33.0,20.0,9.0,9.2420,3555.0,5223.0,101.0,2950.0,4403.0,65.0,605.0,820.0,36.0,0.0,1.0,0.0,0.0,1.0,1.0,5860.0,8207.0,159.0,4751.0,7280.0,120.0,1109.0,927.0,39.0,0.0,1.0,0.0,0.0,1.0,1.0
3,33543,ESPORTSTMNT06/1190148,complete,http://matchhistory.na.leagueoflegends.com/en/...,EM,2020,Spring,0,2020-04-06 15:11:06,1,10.07,4,Blue,bot,Hjarnan,GamerLegion,Varus,Nunu & Willump,Elise,Senna,Akali,Karthus,1649,1,10,2,5,18,7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6549,0.9096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,16036.0,583.4809,0.290613,503.9054,256.7738,9.0,0.3275,5.0,0.1819,2.0,29.0,1.0552,13720,10080.0,366.7677,0.281426,11200.0,NaN,231.0,204.0,27.0,20.0,3.0,8.4051,3825.0,3391.0,76.0,2649.0,2903.0,53.0,1176.0,488.0,23.0,2.0,1.0,2.0,0.0,2.0,2.0,6641.0,6166.0,132.0,4643.0,4679.0,94.0